<a href="https://colab.research.google.com/github/BrunoGomesCoelho/mosquito-networking/blob/master/notebooks_tcc/1.7-BrunoGomesCoelho_Colab_fixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab console code
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
"""

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}setInterval(ClickConnect,60000)\n'

## Check tensorflow version

In [2]:
!python3 -m pip install --upgrade tensorflow -q

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [4]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [5]:
# Start logging time
import time
start_time = time.time()

In [6]:
COLAB_IDX = 1
TEMPERATURE = "t02"
FORMAT = "repeat"

In [7]:
SCALE = True
TESTING = True
COLAB = False

if COLAB:
    BASE_DIR = "/content/drive/My Drive/IC/mosquito-networking/"
else:
    BASE_DIR = "../"

In [8]:
if COLAB:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)

In [9]:
if COLAB:
    import sys
    sys.path.append("/content/drive/My Drive/IC/mosquito-networking/")

In [10]:
if COLAB:
    !python3 -m pip install -qr "/content/drive/My Drive/IC/mosquito-networking/drive_requirements.txt"

In [11]:
if not COLAB:
    # OPTIONAL: Load the "autoreload" extension so that code can change
    %load_ext autoreload
    # OPTIONAL: always reload modules so that as you change code in src, it gets loaded
    %autoreload 2

In [12]:
import numpy as np
import pandas as pd
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()



- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Experiment params

In [13]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'epochs': 2 if TESTING else 100}

# load data

In [14]:
# Load scaler
#scaler = joblib.load("../data/interim/scaler.pkl")
if SCALE:
    scaler = joblib.load(BASE_DIR + 
                         f"data/interim/scaler_{FORMAT}_{TEMPERATURE}_no_std.pkl")

In [15]:
#data = np.load(BASE_DIR + f"data/interim/all_wavs_{FORMAT}_{TEMPERATURE}.npz",
data = np.load(BASE_DIR + f"data/interim/all_wavs_{FORMAT}_{TEMPERATURE}_no_std.npz",
               allow_pickle=True)

In [16]:
SCALE=False

In [17]:
# Parse and divide data
train_data = data["train"]
val_data = data["val"]
test_data = data["test"]

if SCALE:
    x_train = scaler.transform(train_data[:, :-1])
    x_val = scaler.transform(val_data[:, :-1])
    x_test = scaler.transform(test_data[:, :-1])
else:
    x_train = train_data[:, :-1]
    x_val = val_data[:, :-1]
    x_test = test_data[:, :-1]

y_train = train_data[:, -1]
y_val = val_data[:, -1]
y_test = test_data[:, -1]

In [18]:
x_train = x_train / 32
x_val = x_val / 32
x_test = x_test / 32

In [19]:
# Reshape data
x_train = x_train.reshape((*x_train.shape, 1))
x_val = x_val.reshape((*x_val.shape, 1))
x_test = x_test.reshape((*x_test.shape, 1))

In [20]:
# Generators
# I am not using generators since currently the dataset fits on RAM

# Load model

In [21]:
"""A much better way to load a custom module since it allows us to change it easierly
    on the fly. Idea from https://zerowithdot.com/colab-workspace/
"""
from os.path import join
from importlib.machinery import SourceFileLoader
wingbeats_model = SourceFileLoader('wingbeats_model', join(BASE_DIR, 'src/models/wingbeats_model.py')).load_module()
model = wingbeats_model.WingbeatsNetModel(dropout=False)
#from src.models.wingbeats_model import WingbeatsNetModel
#model = WingbeatsNetModel(dropout=False)
# load weights?
#f"runs/colab/{COLAB_IDX-1}/model_epoch_90.pt"))
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise (GaussianNois (None, 11025, 1)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 11023, 16)         64        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11021, 16)         784       
_________________________________________________________________
batch_normalization (BatchNo (None, 11021, 16)         64        
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 929
Trainable params: 897
Non-trainable params: 32
_________________________________________________________

[](wingbeats_model.png)

<br>
<br>
<br>
<br>
<br>
<br>

# Start tensorboard

In [22]:
save_path =  BASE_DIR + f"runs/TCC/{TEMPERATURE}/{FORMAT}/{COLAB_IDX}/"

In [23]:
!python3 -m pip install --upgrade grpcio

  Using cached https://files.pythonhosted.org/packages/30/54/c9810421e41ec0bca2228c6f06b1b1189b196b69533cbcac9f71b44727f8/grpcio-1.24.3-cp36-cp36m-manylinux2010_x86_64.whl
  Found existing installation: grpcio 1.11.0
    Uninstalling grpcio-1.11.0:
      Successfully uninstalled grpcio-1.11.0
  Rolling back uninstall of grpcio
  Moving to /home/bcoelho/.local/lib/python3.6/site-packages/grpc/
   from /home/bcoelho/.local/lib/python3.6/site-packages/~rpc
  Moving to /home/bcoelho/.local/lib/python3.6/site-packages/grpcio-1.11.0.dist-info/
   from /home/bcoelho/.local/lib/python3.6/site-packages/~rpcio-1.11.0.dist-info
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/grpcio-1.24.3.dist-info'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [33]:
#%tensorboard --logdir logs

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# train function

In [26]:
MODEL = "testing"
model_name = f"{MODEL}"

In [27]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                        ReduceLROnPlateau, CSVLogger,
                                        TensorBoard)

In [28]:
# Overfit small amount of data
temp = x_train.copy()
temp_y = y_train.copy()

In [29]:
x_train = x_train[:10]
y_train = y_train[:10]

params["epochs"] = 70
params["batch_size"] = 2

model = wingbeats_model.WingbeatsNetModel(dropout=False, batch_norm=False, 
                                          use_noise=False, blocks=5)#lr=0.0001)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 11023, 16)         64        
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 11021, 16)         784       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 11019, 32)         1568      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 11017, 32)         3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5508, 32)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 5506, 64)          6208      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 5504, 64)         

In [30]:
from datetime import datetime

top_weights_path = 'model_' + model_name + '.h5'

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, write_grads=True,
                                   write_graph=True, write_images=True,
                                   histogram_freq=1)

callbacks_list = [
    #ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1,
    #                save_best_only=True, save_weights_only=True),
    #EarlyStopping(monitor='val_accuracy', patience=6, verbose=1),
    tensorboard_callback]#,
    #ReduceLROnPlateau(monitor = 'val_accuracy', factor=0.1, 
    #                  patience=3, verbose=1),
    #CSVLogger('model_' + model_name + '.log')]

histories = model.fit(x_train, y_train, 
          validation_data=[x_val, y_val],
          callbacks=callbacks_list,
          **params)

#model.load_weights(top_weights_path)
#loss, acc = model.evaluate(x_test, y_test)

#print('loss', loss)
#print('Test accuracy:', acc)

Train on 10 samples, validate on 743 samples
Epoch 1/70
10/10 [==============================] - 17s 2s/sample - loss: 0.8090 - accuracy: 0.5000 - val_loss: 0.7306 - val_accuracy: 0.5330
Epoch 2/70
10/10 [==============================] - 16s 2s/sample - loss: 0.7227 - accuracy: 0.5000 - val_loss: 0.6913 - val_accuracy: 0.5330
Epoch 3/70
10/10 [==============================] - 16s 2s/sample - loss: 0.6990 - accuracy: 0.5000 - val_loss: 0.6911 - val_accuracy: 0.5330
Epoch 4/70
10/10 [==============================] - 16s 2s/sample - loss: 0.6883 - accuracy: 0.5000 - val_loss: 0.6911 - val_accuracy: 0.5330
Epoch 5/70
10/10 [==============================] - 15s 2s/sample - loss: 0.6847 - accuracy: 0.5000 - val_loss: 0.6915 - val_accuracy: 0.5330
Epoch 6/70
10/10 [==============================] - 16s 2s/sample - loss: 0.6835 - accuracy: 0.5000 - val_loss: 0.6922 - val_accuracy: 0.5330
Epoch 7/70
10/10 [==============================] - 16s 2s/sample - loss: 0.6830 - accuracy: 0.5000 - v

10/10 [==============================] - 18s 2s/sample - loss: 0.5752 - accuracy: 0.8000 - val_loss: 0.8229 - val_accuracy: 0.5262
Epoch 58/70
10/10 [==============================] - 18s 2s/sample - loss: 0.6094 - accuracy: 0.8000 - val_loss: 0.7910 - val_accuracy: 0.5289
Epoch 59/70
10/10 [==============================] - 18s 2s/sample - loss: 0.6496 - accuracy: 0.8000 - val_loss: 0.7092 - val_accuracy: 0.4993
Epoch 60/70
10/10 [==============================] - 18s 2s/sample - loss: 0.6300 - accuracy: 0.8000 - val_loss: 0.7326 - val_accuracy: 0.5034
Epoch 61/70
10/10 [==============================] - 18s 2s/sample - loss: 0.6430 - accuracy: 0.7000 - val_loss: 1.0005 - val_accuracy: 0.5370
Epoch 62/70
10/10 [==============================] - 18s 2s/sample - loss: 0.6658 - accuracy: 0.7000 - val_loss: 0.7113 - val_accuracy: 0.5020
Epoch 63/70
10/10 [==============================] - 19s 2s/sample - loss: 0.6690 - accuracy: 0.5000 - val_loss: 0.7002 - val_accuracy: 0.4993
Epoch 64/70

In [31]:
def check_grad_proportion(model, x_train, layer_idx=0):
    name = model.get_config()["layers"][layer_idx]["config"]["name"]
    inputs = tf.cast(x_train, tf.float32)

    with tf.GradientTape() as tape:
        preds = model(inputs)

    grads = tape.gradient(preds, model.get_layer(name).trainable_variables)

    weights =  model.get_layer(name).weights
    proportion = grads[0] / weights[0]
    proportion_avg = tf.math.reduce_mean(proportion)
    print(f"Average proportion between grad and weights: {proportion_avg}")
    print(f"Some proportions:")
    print(proportion[:5])
    return grads, weights


In [32]:
grads, weights = check_grad_proportion(model, x_train)

Average proportion between grad and weights: -7.780606746673584
Some proportions:
tf.Tensor(
[[[  10.691901      0.97272104    5.944378     -2.579638     -3.9449556
     -3.969139      1.6332794     3.77995     -20.242613     -1.7195848
     -2.608939     -0.8888121     2.4468682     6.606426      6.1104445
      2.7996278 ]]

 [[  -2.396848     -1.8808111     3.350695     -4.752765      2.761866
      0.51001793   -3.7456138     2.616417      5.860441     -1.5028809
      1.236941    -10.77132     -13.112622      5.442507     -2.6337285
     -6.1468344 ]]

 [[   2.685281      2.3507676     5.350729      2.0655186    -9.695396
      0.35927266   -1.9425626  -232.74925    -142.08527       1.1353333
      0.5862385    -0.66001636   -8.713153     18.610273     -3.3029807
     12.668719  ]]], shape=(3, 1, 16), dtype=float32)
